In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scripts.model import BiLSTM
import torch as th
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup
from scripts.dataset import create_data_loader
from scripts.model_utils import train, train_epoch, eval_model
from scripts.data_prepare import aggregate_tweets_on_user_level
BEST_MODEL_EN = 'best_model_state_en.bin'
BEST_MODEL_ES = 'best_model_state_es.bin'

Using backend: pytorch


In [2]:
language = input("Please input 'en' for English or 'es' for Spanish:")
if language == 'en':
    best_model_path = BEST_MODEL_EN
else:
    best_model_path = BEST_MODEL_ES

Please input 'en' for English or 'es' for Spanish: en


In [3]:
df = aggregate_tweets_on_user_level()
model = BiLSTM()

In [4]:
from transformers import DistilBertTokenizer
from transformers import DistilBertModel, DistilBertConfig
url = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
tokenizer = DistilBertTokenizer.from_pretrained(url)

In [5]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42
MAX_LEN = 256

In [6]:
df_train, df_test = train_test_split(
  df,
  test_size=0.2,
  random_state=RANDOM_SEED
)

df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=RANDOM_SEED
)

In [7]:
BATCH_SIZE = 16
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [8]:
EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
loss_fn = nn.BCELoss()

In [9]:
# model training
train(language,
          df_train,
          df_val,
          train_data_loader,
          val_data_loader,
          model,
          EPOCHS,
          loss_fn,
          optimizer,
          scheduler)

Epoch 1/10
----------
Train loss 0.6969075500965118 accuracy 0.525
Val   loss 0.6978372037410736 accuracy 0.55

Epoch 2/10
----------
Train loss 0.7055942058563233 accuracy 0.525
Val   loss 0.6911990940570831 accuracy 0.55

Epoch 3/10
----------
Train loss 0.6937666714191437 accuracy 0.525
Val   loss 0.6912884712219238 accuracy 0.55

Epoch 4/10
----------
Train loss 0.6940553724765778 accuracy 0.525
Val   loss 0.6912746131420135 accuracy 0.55

Epoch 5/10
----------
Train loss 0.6918606400489807 accuracy 0.525
Val   loss 0.6912793517112732 accuracy 0.55

Epoch 6/10
----------
Train loss 0.6926493763923645 accuracy 0.525
Val   loss 0.691266655921936 accuracy 0.55

Epoch 7/10
----------
Train loss 0.6928125500679017 accuracy 0.525
Val   loss 0.6912993788719177 accuracy 0.55

Epoch 8/10
----------
Train loss 0.6910923421382904 accuracy 0.525
Val   loss 0.69130539894104 accuracy 0.55

Epoch 9/10
----------
Train loss 0.6923641502857208 accuracy 0.525
Val   loss 0.6913053095340729 accuracy 0

In [10]:
model = BiLSTM()
model.load_state_dict(th.load(BEST_MODEL_EN))

<All keys matched successfully>

In [11]:
from scripts.model_utils import get_predictions
from sklearn.metrics import confusion_matrix, classification_report
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(model, test_data_loader)

In [12]:
print(classification_report(y_test, y_pred, target_names=['0','1']))

              precision    recall  f1-score   support

           0       0.25      1.00      0.40         5
           1       0.00      0.00      0.00        15

    accuracy                           0.25        20
   macro avg       0.12      0.50      0.20        20
weighted avg       0.06      0.25      0.10        20



D:\Anaconda\envs\base2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\envs\base2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\envs\base2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
